In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import numpy as np
from sklearn.model_selection import train_test_split

# Classifying Health Status 


This notebook:

* Builds a characterization of a member’s health status based on their ccs diagnosis data.

* Predicts this health status from a new member's prescription drug data alone

## Exploring the Data

Note: This whole notebook runs in around 10 minutes on my Macbook Air

In [2]:
path = '/Users/gabrielalon/medical'
ccs = pd.read_csv(f"{path}/ccs.csv")
claim_lines = pd.read_csv(f"{path}/claim_lines.csv")
prescription_drugs = pd.read_csv(f"{path}/prescription_drugs.csv")

#### Claim Lines
Models the diagnosis codes (ICD-10) that are present on claims for
medical procedures and services. 

Every row lists one diagnosis given to a member on a certain day.

In [3]:
claim_lines.head(1) #checked more earlier
#claim_lines.tail()
#claim_lines.sample(5)
#claim_lines.isna().sum()
# #24 Nans for the date column found

,record_id,member_id,date_svc,diag1
0,57738,M0000001,2015-12-06,N92.6


#### (CCS): A mapping of diagnosis codes to clinical categories : 

Diagnosis codes found on claim lines are mapped
to higher level clinical categories. We use multi-level CCS. 

The diagnosis code formatting is slightly different in the outpatient claim lines and in this mapping, so we will correct for that

 Not all diagnosis codes have a matching CCS code

In [4]:
ccs.head(1) #checked more earlier
#ccs.tail()
#ccs.sample(5)
#cs.isna().sum() #No Nans found
#ccs['diag'].head(15)

,diag,diag_desc,ccs_1_desc,ccs_2_desc,ccs_3_desc
0,A000,"Cholera due to Vibrio cholerae 01, biovar chol...",Diseases of the digestive system,Intestinal infection [135.],Intestinal infection


#### (NDC): Prescription drug data: 

Every entry in this data set corresponds to a drug prescription filled by a member.

The drugs are identified by their National Drug Code (NDC).

The table provides additional information about which drug category, drug group and drug class a specific drug belongs within

In [5]:
prescription_drugs.head(1) #checked more earlier
#prescription_drugs.tail()
#prescription_drugs.sample(5)
#prescription_drugs.isna().sum() #no nans found

,record_id,member_id,date_svc,ndc,drug_category,drug_group,drug_class
0,4115084976453758912,M0023556,2016-05-08,51285040702,Estrogens,Estrogens,Estrogens


## Data Processing

1. Join claim_lines and ccs and account for " Not all diagnosis codes have a matching CCS code"

2. Then join this with prescription_drugs on member_id

    Then we will have each member_id with a diagnosis, more clinical info on that diagnosis, their prescription drug information, dates for their claim and prescription, etc.

In [6]:
#Data Processing 1

#Removoving the period "." increased exact matches from 154584 to 1917893

#before_fix = claim_lines.merge(ccs, left_on='diag1', right_on='diag',how='inner')
#print(len(before_fix)) #154584
claim_lines['diag1'] = claim_lines['diag1'].str.replace('.', '', regex=False)

In [7]:
claim_and_clinical = claim_lines.merge(ccs, left_on='diag1', right_on='diag',how='inner')
#print(len(claim_and_clinical)) #1917893
#claim_and_clinical.isna().sum() #23 date_svc are missing
claim_and_clinical.head(1) 


,record_id,member_id,date_svc,diag1,diag,diag_desc,ccs_1_desc,ccs_2_desc,ccs_3_desc
0,57738,M0000001,2015-12-06,N926,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders


In [8]:
#Data Processing 2

claim_clinical_prescription = claim_and_clinical.merge(prescription_drugs, on='member_id',how='inner')
claim_clinical_prescription.rename(columns={'date_svc_x': 'claim_date','date_svc_y':'prescription_date','record_id_x':'record_id_claim','record_id_y':'record_id_prescription'}, inplace=True)
#I confirmed that the columns were renamed appropriately with a manual check from the prescription data 
#source and the claim data source

In [9]:
claim_clinical_prescription.head(1)
print(len(claim_clinical_prescription)) #57,299,563
print(len(claim_clinical_prescription['member_id'].unique()))

57299563
194272


After joining the data, there are 194,272 unique member_id values and 57 Million Records

In [10]:
#claim_clinical_prescription.isna().sum()
#claim_date: 968 missing    

### Interpreting the Data for a Single Member

##### Member M0000001 has only one prescription record, which of 15 claims should we associate it with?


The prescription is for herpes but the medical records seem pregnancy related. I am not sure
if there should be a direct link between these.

**To automate** this we can use the aprori algorithm to find likely rules based
 relationships between each  prescription and possible medical condition that it
 should be linked to. This will be imperfect but useful

In [11]:
prescription_drugs[prescription_drugs['member_id'] == 'M0000001']

,record_id,member_id,date_svc,ndc,drug_category,drug_group,drug_class
397235,1990699168614991881,M0000001,2016-03-25,378427593,Antivirals,Herpes Agents,Herpes Agents - Purine Analogues


In [12]:
#This person has 15 claim records and 6 unique diagnosis codes
#6 unique diag_desc although all could be related to pregnancy
#ccs_1_desc had 3 unique, ccs_2_desc had 4 unique, ccs_3_desc had 3 unique

member = 'M0000001'
evaluate_member = claim_and_clinical[claim_and_clinical['member_id'] == member ]
unique_claim_records = (len(evaluate_member['record_id'].unique())) #15
print(f"{unique_claim_records} unique_claim_records for {member}")
#evaluate_member['diag'].unique() #6 unique diagnosis codes
#evaluate_member['diag_desc'].unique() #6 unique diag_desc although all could be related to pregnancy
#evaluate_member['ccs_1_desc'].unique()  #3 unique
#evaluate_member['ccs_2_desc'].unique()  #4 unique
#evaluate_member['ccs_3_desc'].unique()  #3 unique
#evaluate_member #uncomment to see more about this member

15 unique_claim_records for M0000001


### Finding Relevant Patterns: Apriori Algorithm

A test set is held out for later. Also, a random sample of training records is set for improved efficiency.

In [13]:
claim_clinical_prescription, test_df = train_test_split(claim_clinical_prescription, test_size=0.05, random_state=42)

total_training_rows =10000
min_samples = 10 #min count for antecedent and consequent showing up together in the data
min_confidence_ratio = 0.001 #probability of seeing the consequent given the antecedent being present 

In [14]:
#Apriori Algorithm

#its more effective between ndc and ccs_1_desc than ndc and diag
def compute_apriori_rules(total_training_rows=total_training_rows,min_samples=min_samples,min_confidence_ratio=min_confidence_ratio,description_column ='ccs_1_desc',random_state =42):
    min_support = min_samples/total_training_rows #this is the min proportion of samples for a pattern to show up.
    sample_data = claim_clinical_prescription.sample(total_training_rows, random_state =random_state)
    sample_apriori = sample_data.dropna()
    columns = ['ndc','ccs_1_desc'] #diag
    df_onehot = pd.get_dummies(sample_apriori[columns], columns=columns)
    frequent_itemsets = apriori(df_onehot, min_support=min_support, use_colnames=True) 
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence_ratio)

    rules['count'] = rules['support'].apply(lambda x: round(total_training_rows*x))
    rules['count_antec'] = rules['antecedent support'].apply(lambda x: round(total_training_rows*x))
    rules['count_conseq'] = rules['consequent support'].apply(lambda x: round(total_training_rows*x))

    rules = rules[['antecedents', 'consequents','count_antec','count_conseq','count',
        'confidence']] #'lift', 'leverage','conviction', 'zhangs_metric' removed for simplicity
    return rules,test_df

In [15]:
rules,test_df = compute_apriori_rules()
#rules.head(2)
#print(len(rules)) 
#12 rules when total_training_rows =10000, 80 rules when 20000, 238 rules when 30000 (2 min 3 seconds on laptop)

In [16]:
rules_larger,test_df =compute_apriori_rules(total_training_rows=30000,min_samples=min_samples,min_confidence_ratio=min_confidence_ratio)
len(rules_larger)

272

In [17]:
#filter out when the disease ccs is the antecedent
def check_ndc_in_frozenset(fs):
    for elem in fs:
        if 'ndc' in elem:
            return True
    return False

rules_larger = rules_larger[rules_larger['antecedents'].apply(check_ndc_in_frozenset)]

##### Useful Definitions: [Source Link](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/)
**antecedent** is hypothesized to occur before the **consequent**. But here its sufficient to just use the relationships learned without having this as a hard assumption

**confidence** is the probability of seeing the consequent (ccs disease) in a transaction given that it also contains the antecedent (ndc drug code)

In [18]:
rules_larger.head(3)

,antecedents,consequents,count_antec,count_conseq,count,confidence
1,(ndc_2446230),(ccs_1_desc_Diseases of the musculoskeletal sy...,46,4619,10,0.217391
3,(ndc_2751001),(ccs_1_desc_Endocrine; nutritional; and metabo...,22,2588,10,0.454545
5,(ndc_2879959),(ccs_1_desc_Diseases of the musculoskeletal sy...,45,4619,15,0.333333


#### Summary Table of the Rules Learned by Apriori

In [19]:
rules_larger.describe()

,count_antec,count_conseq,count,confidence
count,136.000000,136.000000,136.000000,136.000000
mean,76.845588,3638.110294,13.551471,0.223967
std,52.391246,971.993863,5.872497,0.109538
min,16.000000,810.000000,10.000000,0.046083
25%,44.750000,2797.000000,10.000000,0.144410
50%,60.000000,3984.000000,11.000000,0.197183
75%,86.000000,4619.000000,15.000000,0.275159
max,217.000000,4619.000000,57.000000,0.625000


### Observing Outlier Behaviors

The following table highlights high count antecedents (ccs disease code) that had a low probability ("confidence")
of that consequent (ndc drug code) occuring with it. The total times this occured is in the "count" column


**Example Use Case 1**: Finding **members that are not receiving their required medical care**

AND 

**Example Use Case 2**: Finding **members with unusual care patterns**


We can filter the apriori table by their diagnosis, and then we can compare their prescriptions to those found by the apriori algorithm. The apriori algorithm could discover that the prescription has never been recommended for that diagnosis, or that it is rarely given.

In [20]:
rules_larger = rules_larger.sort_values(['count_antec', 'confidence'], ascending =[False, True])
rules_larger.head(5)

,antecedents,consequents,count_antec,count_conseq,count,confidence
197,(ndc_59310057922),(ccs_1_desc_Neoplasms),217,1723,10,0.046083
187,(ndc_59310057922),(ccs_1_desc_Diseases of the digestive system),217,1577,11,0.050691
194,(ndc_59310057922),(ccs_1_desc_Injury and poisoning),217,1356,11,0.050691
191,(ndc_59310057922),(ccs_1_desc_Diseases of the nervous system and...,217,2069,17,0.078341
184,(ndc_59310057922),(ccs_1_desc_Diseases of the circulatory system),217,3231,19,0.087558


## Retrieving Health Status

In [21]:
all_medical_descriptions = claim_clinical_prescription.groupby('member_id')['ccs_1_desc'].unique().reset_index()

In [22]:
def all_health_status(whole_df=all_medical_descriptions):
    # Creating health description per member
    def create_description(row):
        # Filter out empty strings and 'nan' strings
        filtered_issues = list(filter(lambda x: x not in ('', 'nan'), row['ccs_1_desc']))
        
        if len(filtered_issues) == 0:
            return f"Member {row['member_id']} has no health issues recorded."
        
        else:
            health_issues = ', '.join(filtered_issues).lower()
            return f"Member {row['member_id']} had {health_issues}"

    all_medical_descriptions['high_level_desc'] = all_medical_descriptions.apply(create_description, axis=1)
    return all_medical_descriptions

In [23]:
health_status_df = all_health_status(all_medical_descriptions)

In [24]:
pd.set_option('display.max_colwidth', None)
health_status_df.head(2)

,member_id,ccs_1_desc,high_level_desc
0,M0000001,"[Certain conditions originating in the perinatal period, Diseases of the genitourinary system, Complications of pregnancy; childbirth; and the puerperium]","Member M0000001 had certain conditions originating in the perinatal period, diseases of the genitourinary system, complications of pregnancy; childbirth; and the puerperium"
1,M0000002,"[Symptoms; signs; and ill-defined conditions and factors influencing health status, Diseases of the circulatory system, Residual codes; unclassified; all E codes [259. and 260.], Mental Illness]","Member M0000002 had symptoms; signs; and ill-defined conditions and factors influencing health status, diseases of the circulatory system, residual codes; unclassified; all e codes [259. and 260.], mental illness"


In [25]:
print(f"Above the health_status_df has the health status of {len(health_status_df)} members")

Above the health_status_df has the health status of 193794 members


#### Find Health Status by Single Member ID

In [26]:
single_member_id = 'M0000001' ###Can Change it here

In [27]:
def find_individual_member_health_status(member_id, health_desc_df):
    try:
        print(health_desc_df[health_status_df['member_id'] == member_id]['high_level_desc'].iloc[0])
    except:
        print("Member results not found")
    
find_individual_member_health_status(single_member_id,health_status_df)

Member M0000001 had certain conditions originating in the perinatal period, diseases of the genitourinary system, complications of pregnancy; childbirth; and the puerperium


In [28]:
#Uncomment to see the full records for this person
#claim_clinical_prescription[claim_clinical_prescription['member_id'] ==single_member_id] 

***

## Predicting Health Status for "New Members"

In [29]:
#We are imagining these are "new members"
sample_prescriptions = prescription_drugs.sample(1000, random_state=42)
sample_prescriptions.head(2)

,record_id,member_id,date_svc,ndc,drug_category,drug_group,drug_class
1873473,8069952898672096371,M0160460,2016-12-05,93002601,Antihypertensives,Ace Inhibitors,Ace Inhibitors
576971,7403202619934856205,M0173396,2016-04-22,65862001501,Penicillins,Aminopenicillins,Aminopenicillins


In [30]:
#filtering apriori results for the ndc prescription list of the new member(s)
def filter_apriori(rules, ndc_list):
    all_ndc = ndc_list
    all_ndc = [frozenset({f"ndc_{x}"}) for x in all_ndc]
    return rules[rules['antecedents'].isin(all_ndc)]

Demonstrating retrieving all of the learned rules for a particular drug(s) as the antecedent

In [31]:
test_drug_ndc = ['59310057922','65862001501']
rules_table = filter_apriori(rules_larger, test_drug_ndc)
rules_table.head(2) #remove .head to see more, #its confirmed this works for multiple drugs

,antecedents,consequents,count_antec,count_conseq,count,confidence
197,(ndc_59310057922),(ccs_1_desc_Neoplasms),217,1723,10,0.046083
187,(ndc_59310057922),(ccs_1_desc_Diseases of the digestive system),217,1577,11,0.050691


In [32]:
def predict_health_status_from_prescriptions(rules=rules_larger, new_member_prescription_df=None):
    drug_list = new_member_prescription_df['ndc'].tolist()
    rules_table = filter_apriori(rules_larger, drug_list)
    consequents = rules_table['consequents']
    new_member_prescription_df['ccs_1_desc']  = consequents
    results= all_health_status(new_member_prescription_df)
    return results

In [33]:
new_members_health_status = predict_health_status_from_prescriptions(new_member_prescription_df=sample_prescriptions)

In [34]:
new_members_health_status.head(2)

,member_id,ccs_1_desc,high_level_desc
0,M0000001,"[Certain conditions originating in the perinatal period, Diseases of the genitourinary system, Complications of pregnancy; childbirth; and the puerperium]","Member M0000001 had certain conditions originating in the perinatal period, diseases of the genitourinary system, complications of pregnancy; childbirth; and the puerperium"
1,M0000002,"[Symptoms; signs; and ill-defined conditions and factors influencing health status, Diseases of the circulatory system, Residual codes; unclassified; all E codes [259. and 260.], Mental Illness]","Member M0000002 had symptoms; signs; and ill-defined conditions and factors influencing health status, diseases of the circulatory system, residual codes; unclassified; all e codes [259. and 260.], mental illness"


### Here we go!
We can look look at a member id from our sample and
see if it can come up with sensible health conditions

In [35]:
single_member_id = 'M0000001'

In [36]:
find_individual_member_health_status(single_member_id,new_members_health_status)
#claim_clinical_prescription[claim_clinical_prescription['member_id']==single_member_id] #uncomment to see their original record

Member M0000001 had certain conditions originating in the perinatal period, diseases of the genitourinary system, complications of pregnancy; childbirth; and the puerperium


### Testing Effectiveness on Held-Out Test Set

A held-out test set was created before training the apriori algorithm.

We can predict their health outcomes here (with limited quality due to training time)

In [37]:
final_test_df_inputs = test_df.drop(['record_id_claim','claim_date','diag1','diag','diag_desc','ccs_1_desc','ccs_2_desc','ccs_3_desc'],axis=1)
final_test_df_inputs.head()

,member_id,record_id_prescription,prescription_date,ndc,drug_category,drug_group,drug_class
54509709,M0195787,8283166642040293454,2016-05-12,10702005601,Analgesics - Opioid,Opioid Agonists,Opioid Agonists
10938812,M0131375,5229971531470864098,2015-12-30,16252057350,Antihypertensives,Ace Inhibitors,Ace Inhibitors
35274986,M0028594,7870468309801513981,2016-09-09,59417010310,Adhd/Anti-narcolepsy/Anti-obesity/Anorexiants,Amphetamines,Amphetamines
29076468,M0265600,4511063108344509296,2016-04-22,50111064801,Antidepressants,Selective Serotonin Reuptake Inhibitors (SSRIs),Selective Serotonin Reuptake Inhibitors (SSRIs)
20546696,M0267115,1069932649445796055,2015-09-09,93746598,Antihypertensives,Angiotensin Ii Receptor Antagonists,Angiotensin Ii Receptor Antagonists


In [38]:
new_members_health_status = predict_health_status_from_prescriptions(new_member_prescription_df=final_test_df_inputs)

The function below selects a literally random member from the held-out test set

In [39]:
random_test_member = final_test_df_inputs['member_id'].sample(1).iloc[0]
find_individual_member_health_status(random_test_member,new_members_health_status)

Member M0230246 had symptoms; signs; and ill-defined conditions and factors influencing health status, diseases of the musculoskeletal system and connective tissue, mental illness, neoplasms, infectious and parasitic diseases


Here you can see the original held out test data for the prediction above

In [40]:
test_df[test_df['member_id']==random_test_member][['member_id','ccs_1_desc','ccs_2_desc','ccs_3_desc']].head()

,member_id,ccs_1_desc,ccs_2_desc,ccs_3_desc
17776284,M0230246,Symptoms; signs; and ill-defined conditions and factors influencing health status,Factors influencing health care,Medical examination/evaluation
17776762,M0230246,Mental Illness,Anxiety disorders [651],Anxiety disorders
17777172,M0230246,Diseases of the musculoskeletal system and connective tissue,Other bone disease and musculoskeletal deformities [212.],Other bone disease and musculoskeletal deformities
17776708,M0230246,Symptoms; signs; and ill-defined conditions and factors influencing health status,Factors influencing health care,Other screening for suspected conditions (not mental disorders or infectious disease)
17777334,M0230246,Neoplasms,Benign neoplasms,Other and unspecified benign neoplasm


## Future Work

This approach **could greatly improve with more compute power**.

A summary shown earlier showed that the mean "confidence" of the larger rules dataframe was roughly 0.22, with a standard deviation around 0.1 while the max was roughly 0.62 (where the best is 1). However, I only trained on 30,000 rows out of 54,434,584. This was for the sake of efficiency (and not crashing my laptop!). The algorithm only learned about 136 rules under these constraints.

Several more improvements would be made: 

* The rules that are applied to make the predictions can be filtered by several metrics to reduce false positives. These include the count of the final predictions, the count of the antecedent (ccs of the disease),	the count of the consequent (ndc prescription code), the joint count, the	confidence (probability of the consequent given the antecedent).

* It may help to narrow the window of time between a claim date and a prescription date for a member such that unjustifiable associations would not be analyzed by the apriori algorithm. 

* Find out if there is a gaurantee that the claim date will always be after the prescription date for that condition that is justifying the prescription.

* Evaluate over a large test set and also prepare it better so that the train and test data don't have the same member ids show up in both train and test (to avoid data leakage)

* The final model can be selected based on the confusion matrix values for true positive, false positive, false negative, true negative and related metrics. 

* When evaluating over a large test set try exact match scoring, but also text similarity scoring like the BLEU score, and
sentence similarity with LLM transformers like BERT from the HuggingFace library.

***

Outside data that could help:

* A rules based dataset for what can be prescribed may already exist and be regulated by the government?

* Existing medical knowledge about disease progression and what is prescribed at each stage of the disease can help predict
  the prescriptions that may be taken as a function of time

* The inclusion of demographic data or other data about members' background and health could also help for rationalized and predicting what will be prescribed for each condition.